In [2]:
import sqlite3 as sql

class ParkingArea:

    def __init__(self,parking_area_name : str,total_capacity : int, currently_filled = 0 ) -> None:

        # Parameter initialization
        self.name = parking_area_name
        self.total_capacity = total_capacity
        self.currently_filled = currently_filled
        self.avg_occupancy = 0
        self.avg_new_visiting = 0

        # Database Connetion Initialization
        self._db_name = parking_area_name +  ".db"
        self.parked_table_name = parking_area_name+ "_parked"
        self.record_table_name = parking_area_name+ "_record"
        self.customer_table_name = parking_area_name+ "_customer"

        # Database Creation
        self._db = sql.connect(self._db_name)
        
        # Database Cursor Declaration
        self.db_cur = self._db.cursor()
        self.db_cur.execute(f"CREATE TABLE {self.parked_table_name} (Car_Number VARCHAR , In_Time VARCHAR)")
        self.db_cur.execute(f"CREATE TABLE {self.record_table_name} (Car_Number VARCHAR , In_Time VARCHAR , Out_Time VARCHAR)")
        self.db_cur.execute(f"CREATE TABLE {self.customer_table_name} (Car_Id INTEGER, Car_Number VARCHAR)")
    
    def customer_data_entry(self, vehicle_id : int, vehicle_number : str) -> bool:

        #Adding Customer's Id and Car Number
        try:
            cmd = f'''INSERT INTO {self.customer_table_name} VALUES ( "{vehicle_id}", "{vehicle_number}")'''
            self.db_cur.execute(cmd)
            return True

        except:
            print("Entry failed")
            return False
        

    def entry(self, vechile_id : str, In_Time : str) -> bool:
        '''
            Adds the car to Parked Table
        '''
        flag = 0
        try:
            #Check if slots are available
            if self.currently_filled < self.total_capacity:
                cmd = f'''INSERT INTO {self.parked_table_name} VALUES ("{vechile_id}","{In_Time}")'''
                self.db_cur.execute(cmd)
                self.currently_filled+=1
                flag = 1
            else:

                print("No free parking slots available")
                return False
            return True
        
        except:
            print("Entry Failed")
            return False
    
    
    def exit(self, vechile_id : str , Out_Time : str) -> bool:
        '''
            Remove the car from the Parked Database and Its to the Entry Records 
        '''
        try:
            select_cmd = f"SELECT * FROM {self.parked_table_name} WHERE Car_Number = '{vechile_id}'"
            car_data_cur   = self.db_cur.execute(select_cmd)
            vechile_id,In_Time = car_data_cur.fetchone()
            
            push_data = f'''INSERT INTO {self.record_table_name} VALUES ("{vechile_id}","{In_Time}","{Out_Time}")'''
            self.db_cur.execute(push_data)

            cmd = f"DELETE FROM {self.parked_table_name} WHERE Car_Number = '{vechile_id}'"
            self.db_cur.execute(cmd)
            self.currently_filled-=1

            return True
        except:
            print("Exit Failed")
            return False

In [1]:
import pandas as pd
sample_data = pd.read_csv('entry_exit data/entry_exit_2.csv')
customer_data = pd.read_csv('customer_data/customer_data.csv')

In [3]:

vehicle_id = sample_data['vehicle_id']
vehicle_number = sample_data['vehicle_license_plate']
entry_time_data = sample_data['entry_time']
exit_time_data = sample_data['exit_time']

cust_id = customer_data['car_id']
cust_number = customer_data['car_number']
print(len(cust_id))

20


In [4]:
sample_area = ParkingArea('Sample_mall',20)
sample_area.parked_table_name

'Sample_mall_parked'

In [5]:
#QUICK COMMANDS FOR RESETTING TABLE
del_cmd = f"DELETE FROM {sample_area.parked_table_name}"
del_cmd2 = f"DELETE FROM {sample_area.record_table_name}"
sample_area.db_cur.execute(del_cmd)
sample_area.db_cur.execute(del_cmd2)


In [7]:
for i in range(0,len(cust_id)):
    id = cust_id[i]
    number = cust_number[i]
    sample_area.customer_data_entry(id,number)

In [8]:
for i in range(0,len(vehicle_id)):   
    id = vehicle_id[i] 
    number = vehicle_number[i]
    entry = entry_time_data[i]
    exit = exit_time_data[i]

    cmd = f"SELECT * FROM {sample_area.parked_table_name} WHERE Car_Number = '{number}'"
    check = sample_area.db_cur.execute(cmd).fetchall()

    if len(check) == 0:
        ret = sample_area.entry(number,entry)
        print(f"Vehicle Id = {id}  {ret}  ")
    else:
        ret = sample_area.exit(number,exit)
    
    
    

    

Vehicle Id = 1  True  
Vehicle Id = 2  True  
Vehicle Id = 3  True  
Vehicle Id = 4  True  
Vehicle Id = 5  True  
Vehicle Id = 11  True  
Vehicle Id = 12  True  
Vehicle Id = 13  True  
Vehicle Id = 14  True  
Vehicle Id = 15  True  
Vehicle Id = 16  True  
Vehicle Id = 17  True  
Vehicle Id = 18  True  
Vehicle Id = 19  True  
Vehicle Id = 20  True  
Vehicle Id = 21  True  
Vehicle Id = 22  True  
Vehicle Id = 23  True  
Vehicle Id = 24  True  
Vehicle Id = 25  True  
Vehicle Id = 26  True  
Vehicle Id = 27  True  
Vehicle Id = 28  True  
Vehicle Id = 29  True  
Vehicle Id = 30  True  
Vehicle Id = 46  True  
Vehicle Id = 47  True  
Vehicle Id = 48  True  
Vehicle Id = 49  True  
Vehicle Id = 50  True  
Vehicle Id = 51  True  
Vehicle Id = 52  True  
Vehicle Id = 53  True  
Vehicle Id = 54  True  
Vehicle Id = 55  True  
Vehicle Id = 56  True  
Vehicle Id = 57  True  
Vehicle Id = 58  True  
Vehicle Id = 59  True  
Vehicle Id = 60  True  
No free parking slots available
Vehicle Id = 

In [11]:
print(sample_area.db_cur.execute(f"SELECT * FROM {sample_area.parked_table_name}").fetchall())


[('KA-01-AB-1234', '09:00'), ('DL-05-CD-5678', '09:30'), ('MH-12-EF-9101', '10:00'), ('AP-31-GH-2345', '10:30'), ('TN-22-IJ-6789', '11:00'), ('KA-05-GH-6789', '09:00'), ('DL-09-IJ-1234', '09:30'), ('MH-16-KL-5678', '10:00'), ('AP-35-MN-9101', '10:30'), ('TN-26-OP-2345', '11:00'), ('KA-06-QR-3456', '09:00'), ('DL-10-ST-7890', '09:30'), ('MH-17-UV-1234', '10:00'), ('AP-36-WX-5678', '10:30'), ('TN-27-YZ-9101', '11:00'), ('KA-07-AB-1234', '09:00'), ('DL-11-CD-5678', '09:30'), ('MH-18-EF-9101', '10:00'), ('AP-37-GH-2345', '10:30'), ('TN-28-IJ-6789', '11:00')]
